In [29]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
from torchvision.models import mobilenet_v2, densenet121

torch.__version__

'2.0.0+cu117'

In [30]:
from torchvision import transforms

batch_size = 1
img_size = 224

train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.Resize(size=(64,64)),
    transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
    transforms.Resize(size=(64,64)),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.ToTensor()
    ])

In [31]:
from torchvision import datasets
from torch.utils.data import DataLoader

train_set = datasets.ImageFolder("./dataset/train", transform=train_transform)
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)

test_set = datasets.ImageFolder("./dataset/test", transform=test_transform)
testloader = DataLoader(test_set, shuffle=False)

In [32]:
feature, target = next(iter(trainloader))
feature.shape

torch.Size([1, 3, 224, 224])

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [34]:
model = mobilenet_v2(num_classes=2).to(device)
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [35]:
# model.classifier[1] = nn.Linear(model.last_channel, 2)
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [36]:
from torchinfo import summary

summary(model, input_size=[8,3,64,64])

c:\Users\siddh\AppData\Local\Programs\Python\Python39\lib\site-packages\torchinfo\torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
c:\Users\siddh\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV2                                        [8, 2]                    --
├─Sequential: 1-1                                  [8, 1280, 2, 2]           --
│    └─Conv2dNormActivation: 2-1                   [8, 32, 32, 32]           --
│    │    └─Conv2d: 3-1                            [8, 32, 32, 32]           864
│    │    └─BatchNorm2d: 3-2                       [8, 32, 32, 32]           64
│    │    └─ReLU6: 3-3                             [8, 32, 32, 32]           --
│    └─InvertedResidual: 2-2                       [8, 16, 32, 32]           --
│    │    └─Sequential: 3-4                        [8, 16, 32, 32]           896
│    └─InvertedResidual: 2-3                       [8, 24, 16, 16]           --
│    │    └─Sequential: 3-5                        [8, 24, 16, 16]           5,136
│    └─InvertedResidual: 2-4                       [8, 24, 16, 16]           --
│    │    └─Sequential: 3-6   

In [37]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

In [38]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(trainloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        # outputs = torch.squeeze(outputs)
        # outputs = outputs.to(torch.float32)
        # labels = labels.to(torch.float32)
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
    # scheduler.step()

    return last_loss

In [39]:
def accuracy(predictions, labels):
    classes = torch.argmax(predictions, dim=1)
    return torch.mean((classes == labels).float())

In [40]:
epoch_number = 0
from datetime import datetime
torch.cuda.manual_seed(42)
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
best_vloss = 1_000_000.
data_for_acc  =[]
EPOCHS = 20
for epoch in range(EPOCHS):
    running_accuracy = 0.00
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number)

    # We don't need gradients on to do reporting
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(testloader):
        vinputs, vlabels = vdata
        vinputs, vlabels = vinputs.to(device), vlabels.to(device)
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        running_accuracy += accuracy(voutputs, vlabels)
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    running_accuracy /= len(testloader)
    print('LOSS train {} test {} accuracy {}'.format(avg_loss, avg_vloss, running_accuracy))

    # Log the running loss averaged per batch
    # for both training and validation
  
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0.0 test 0.6977038383483887 accuracy 0.5041322112083435
EPOCH 2:
LOSS train 0.0 test 0.6976239085197449 accuracy 0.42148759961128235
EPOCH 3:


KeyboardInterrupt: 

In [ ]:
torch.save(model, 'MobileNet.pth')